# Chapter 7-4: 캡슐화와 다형성

## 학습 목표
- 캡슐화의 개념과 중요성 이해하기
- 접근 제어자 사용법 익히기
- 다형성의 개념과 활용법 이해하기
- 메서드 오버라이딩 실습하기

## 1. 캡슐화 (Encapsulation)

캡슐화는 객체의 내부 데이터와 메서드를 외부에서 직접 접근하지 못하도록 숨기는 것입니다.
Python에서는 언더스코어(_)를 사용하여 접근을 제어합니다.

In [ ]:
# 캡슐화 예제 - 은행 계좌 클래스
class BankAccount:
    def __init__(self, account_number, initial_balance=0):
        self.account_number = account_number  # 공개 속성
        self._balance = initial_balance       # 보호된 속성 (protected)
        self.__pin = "1234"                  # 비공개 속성 (private)
    
    # 잔액 조회 메서드 (getter)
    def get_balance(self):
        return self._balance
    
    # 입금 메서드
    def deposit(self, amount):
        if amount > 0:
            self._balance += amount
            print(f"{amount:,}원이 입금되었습니다. 현재 잔액: {self._balance:,}원")
        else:
            print("입금 금액은 0보다 커야 합니다.")
    
    # 출금 메서드
    def withdraw(self, amount, pin):
        if not self._verify_pin(pin):
            print("PIN이 올바르지 않습니다.")
            return False
        
        if amount > self._balance:
            print("잔액이 부족합니다.")
            return False
        
        if amount <= 0:
            print("출금 금액은 0보다 커야 합니다.")
            return False
        
        self._balance -= amount
        print(f"{amount:,}원이 출금되었습니다. 현재 잔액: {self._balance:,}원")
        return True
    
    # 비공개 메서드 - PIN 확인
    def _verify_pin(self, pin):
        return pin == self.__pin
    
    # PIN 변경 메서드
    def change_pin(self, old_pin, new_pin):
        if self._verify_pin(old_pin):
            self.__pin = new_pin
            print("PIN이 성공적으로 변경되었습니다.")
        else:
            print("기존 PIN이 올바르지 않습니다.")

# 계좌 생성 및 테스트
account = BankAccount("123-456-789", 100000)

# 공개 속성 접근
print(f"계좌번호: {account.account_number}")

# 메서드를 통한 안전한 접근
print(f"현재 잔액: {account.get_balance():,}원")
account.deposit(50000)
account.withdraw(30000, "1234")
account.withdraw(30000, "wrong_pin")

## 2. 접근 제어자의 종류

- **public** (공개): 일반적인 속성/메서드
- **protected** (보호): _로 시작하는 속성/메서드 (관례적으로 내부 사용)
- **private** (비공개): __로 시작하는 속성/메서드 (외부 접근 제한)

In [ ]:
# 접근 제어 테스트
print("=== 접근 제어 테스트 ===")

# 공개 속성 접근 - 성공
print(account.account_number)

# 보호된 속성 접근 - 가능하지만 권장하지 않음
print(f"보호된 속성 접근: {account._balance}")

# 비공개 속성 직접 접근 시도 - 오류 발생
try:
    print(account.__pin)
except AttributeError as e:
    print(f"오류: {e}")

# name mangling을 통한 접근 (권장하지 않음)
print(f"Name mangling을 통한 접근: {account._BankAccount__pin}")

## 3. Property 데코레이터

@property 데코레이터를 사용하면 메서드를 속성처럼 사용할 수 있습니다.

In [ ]:
class Temperature:
    def __init__(self, celsius=0):
        self._celsius = celsius
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("절대영도(-273.15°C) 이하는 불가능합니다.")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return (self._celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = (value - 32) * 5/9
    
    @property
    def kelvin(self):
        return self._celsius + 273.15
    
    def __str__(self):
        return f"{self._celsius}°C ({self.fahrenheit:.1f}°F, {self.kelvin:.1f}K)"

# 온도 클래스 테스트
temp = Temperature(25)
print(f"초기 온도: {temp}")

# 속성처럼 접근
print(f"섭씨: {temp.celsius}°C")
print(f"화씨: {temp.fahrenheit:.1f}°F")
print(f"켈빈: {temp.kelvin:.1f}K")

# setter 사용
temp.fahrenheit = 86
print(f"화씨 86도로 설정 후: {temp}")

# 잘못된 값 설정 시도
try:
    temp.celsius = -300
except ValueError as e:
    print(f"오류: {e}")

## 4. 다형성 (Polymorphism)

다형성은 같은 인터페이스를 사용하여 서로 다른 타입의 객체들을 다룰 수 있는 능력입니다.

In [ ]:
# 추상 기본 클래스
class Animal:
    def __init__(self, name, species):
        self.name = name
        self.species = species
    
    def speak(self):
        pass  # 추상 메서드 (서브클래스에서 구현해야 함)
    
    def move(self):
        return f"{self.name}이(가) 움직입니다."
    
    def introduce(self):
        return f"안녕하세요! 저는 {self.species} {self.name}입니다."

class Dog(Animal):
    def __init__(self, name, breed):
        super().__init__(name, "개")
        self.breed = breed
    
    def speak(self):
        return "멍멍!"
    
    def move(self):
        return f"{self.name}이(가) 네 발로 뛰어다닙니다."
    
    def fetch(self):
        return f"{self.name}이(가) 공을 가져옵니다."

class Cat(Animal):
    def __init__(self, name, color):
        super().__init__(name, "고양이")
        self.color = color
    
    def speak(self):
        return "야옹~"
    
    def move(self):
        return f"{self.name}이(가) 우아하게 걸어다닙니다."
    
    def climb(self):
        return f"{self.name}이(가) 나무를 타고 올라갑니다."

class Bird(Animal):
    def __init__(self, name, wing_span):
        super().__init__(name, "새")
        self.wing_span = wing_span
    
    def speak(self):
        return "짹짹!"
    
    def move(self):
        return f"{self.name}이(가) 하늘을 날아다닙니다."
    
    def fly(self):
        return f"{self.name}이(가) {self.wing_span}cm 날개를 펼치고 납니다."

# 동물들 생성
animals = [
    Dog("멍멍이", "골든 리트리버"),
    Cat("야옹이", "검은색"),
    Bird("짹짹이", 30)
]

print("=== 다형성 예제 ===")
for animal in animals:
    print(f"\n{animal.introduce()}")
    print(f"울음소리: {animal.speak()}")
    print(f"움직임: {animal.move()}")

## 5. 메서드 오버라이딩과 super()

메서드 오버라이딩은 부모 클래스의 메서드를 자식 클래스에서 재정의하는 것입니다.

In [ ]:
class Vehicle:
    def __init__(self, brand, model, year):
        self.brand = brand
        self.model = model
        self.year = year
        self.speed = 0
    
    def start_engine(self):
        return f"{self.brand} {self.model}의 엔진이 시동됩니다."
    
    def stop_engine(self):
        self.speed = 0
        return f"{self.brand} {self.model}의 엔진이 정지됩니다."
    
    def accelerate(self, speed_increase):
        self.speed += speed_increase
        return f"속도가 {speed_increase}km/h 증가했습니다. 현재 속도: {self.speed}km/h"
    
    def get_info(self):
        return f"{self.year}년 {self.brand} {self.model}"

class ElectricCar(Vehicle):
    def __init__(self, brand, model, year, battery_capacity):
        super().__init__(brand, model, year)  # 부모 클래스 초기화
        self.battery_capacity = battery_capacity
        self.battery_level = 100
    
    # 메서드 오버라이딩
    def start_engine(self):
        base_message = super().start_engine()  # 부모 메서드 호출
        return f"{base_message} (전기 모터 가동)"
    
    def stop_engine(self):
        base_message = super().stop_engine()
        return f"{base_message} (전기 모터 정지)"
    
    def charge_battery(self, charge_amount):
        old_level = self.battery_level
        self.battery_level = min(100, self.battery_level + charge_amount)
        actual_charge = self.battery_level - old_level
        return f"배터리 {actual_charge}% 충전되었습니다. 현재 배터리: {self.battery_level}%"
    
    def accelerate(self, speed_increase):
        # 배터리 소모 계산
        battery_consumption = speed_increase * 0.1
        if self.battery_level < battery_consumption:
            return "배터리가 부족합니다. 충전이 필요합니다."
        
        self.battery_level -= battery_consumption
        base_message = super().accelerate(speed_increase)
        return f"{base_message} (배터리: {self.battery_level:.1f}%)"
    
    # 메서드 오버라이딩
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} (전기차, 배터리 용량: {self.battery_capacity}kWh)"

class SportsCar(Vehicle):
    def __init__(self, brand, model, year, max_speed):
        super().__init__(brand, model, year)
        self.max_speed = max_speed
        self.turbo_mode = False
    
    def activate_turbo(self):
        self.turbo_mode = True
        return "터보 모드가 활성화되었습니다!"
    
    def deactivate_turbo(self):
        self.turbo_mode = False
        return "터보 모드가 비활성화되었습니다."
    
    # 메서드 오버라이딩
    def accelerate(self, speed_increase):
        if self.turbo_mode:
            speed_increase *= 1.5  # 터보 모드에서 50% 빠른 가속
        
        if self.speed + speed_increase > self.max_speed:
            return f"최대 속도 {self.max_speed}km/h를 초과할 수 없습니다."
        
        base_message = super().accelerate(speed_increase)
        turbo_status = " (터보 모드)" if self.turbo_mode else ""
        return f"{base_message}{turbo_status}"
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} (스포츠카, 최대 속도: {self.max_speed}km/h)"

# 차량 테스트
vehicles = [
    ElectricCar("Tesla", "Model 3", 2023, 75),
    SportsCar("Ferrari", "488 GTB", 2022, 330)
]

print("=== 메서드 오버라이딩 예제 ===")
for vehicle in vehicles:
    print(f"\n=== {vehicle.get_info()} ===")
    print(vehicle.start_engine())
    
    if isinstance(vehicle, ElectricCar):
        print(vehicle.charge_battery(0))  # 현재 배터리 상태 확인
        print(vehicle.accelerate(50))
        print(vehicle.accelerate(100))
    elif isinstance(vehicle, SportsCar):
        print(vehicle.activate_turbo())
        print(vehicle.accelerate(100))
        print(vehicle.deactivate_turbo())
        print(vehicle.accelerate(50))
    
    print(vehicle.stop_engine())

## 6. 다형성을 활용한 실용적 예제

도형 클래스를 만들어 다형성을 활용해보겠습니다.

In [ ]:
import math

class Shape:
    def __init__(self, color="white"):
        self.color = color
    
    def area(self):
        raise NotImplementedError("서브클래스에서 구현해야 합니다.")
    
    def perimeter(self):
        raise NotImplementedError("서브클래스에서 구현해야 합니다.")
    
    def get_info(self):
        return f"{self.color} 색상의 {self.__class__.__name__}"

class Rectangle(Shape):
    def __init__(self, width, height, color="white"):
        super().__init__(color)
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} (가로: {self.width}, 세로: {self.height})"

class Circle(Shape):
    def __init__(self, radius, color="white"):
        super().__init__(color)
        self.radius = radius
    
    def area(self):
        return math.pi * self.radius ** 2
    
    def perimeter(self):
        return 2 * math.pi * self.radius
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} (반지름: {self.radius})"

class Triangle(Shape):
    def __init__(self, base, height, side1, side2, color="white"):
        super().__init__(color)
        self.base = base
        self.height = height
        self.side1 = side1
        self.side2 = side2
    
    def area(self):
        return 0.5 * self.base * self.height
    
    def perimeter(self):
        return self.base + self.side1 + self.side2
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} (밑변: {self.base}, 높이: {self.height})"

# 도형 계산기 함수
def calculate_shape_properties(shapes):
    """
    다형성을 활용한 도형 속성 계산 함수
    """
    total_area = 0
    total_perimeter = 0
    
    print("=== 도형 속성 계산 ===")
    for i, shape in enumerate(shapes, 1):
        area = shape.area()
        perimeter = shape.perimeter()
        
        print(f"\n{i}. {shape.get_info()}")
        print(f"   면적: {area:.2f}")
        print(f"   둘레: {perimeter:.2f}")
        
        total_area += area
        total_perimeter += perimeter
    
    print(f"\n=== 전체 합계 ===")
    print(f"총 면적: {total_area:.2f}")
    print(f"총 둘레: {total_perimeter:.2f}")
    print(f"평균 면적: {total_area/len(shapes):.2f}")
    print(f"평균 둘레: {total_perimeter/len(shapes):.2f}")

# 다양한 도형 생성
shapes = [
    Rectangle(5, 3, "빨간색"),
    Circle(4, "파란색"),
    Triangle(6, 4, 5, 5, "초록색"),
    Rectangle(2, 8, "노란색"),
    Circle(2.5, "보라색")
]

# 다형성을 활용한 계산
calculate_shape_properties(shapes)

## 7. 실습 문제

학습한 내용을 바탕으로 문제를 해결해보세요.

In [ ]:
# 실습 문제 1: 직원 관리 시스템
# Employee 기본 클래스와 Manager, Developer 서브클래스를 구현하세요.
# 요구사항:
# 1. Employee: name, employee_id, base_salary 속성
# 2. calculate_salary() 메서드 (기본급만 반환)
# 3. Manager: bonus_rate 속성, calculate_salary() 오버라이딩 (기본급 + 보너스)
# 4. Developer: overtime_hours, hourly_rate 속성, calculate_salary() 오버라이딩 (기본급 + 야근수당)

class Employee:
    def __init__(self, name, employee_id, base_salary):
        self.name = name
        self.employee_id = employee_id
        self._base_salary = base_salary  # 보호된 속성
    
    @property
    def base_salary(self):
        return self._base_salary
    
    @base_salary.setter
    def base_salary(self, value):
        if value < 0:
            raise ValueError("급여는 0 이상이어야 합니다.")
        self._base_salary = value
    
    def calculate_salary(self):
        return self._base_salary
    
    def get_info(self):
        return f"{self.name} (ID: {self.employee_id})"

class Manager(Employee):
    def __init__(self, name, employee_id, base_salary, bonus_rate=0.2):
        super().__init__(name, employee_id, base_salary)
        self.bonus_rate = bonus_rate
    
    def calculate_salary(self):
        base = super().calculate_salary()
        bonus = base * self.bonus_rate
        return base + bonus
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} - 매니저 (보너스율: {self.bonus_rate*100}%)"

class Developer(Employee):
    def __init__(self, name, employee_id, base_salary, hourly_rate=50000):
        super().__init__(name, employee_id, base_salary)
        self.hourly_rate = hourly_rate
        self.overtime_hours = 0
    
    def add_overtime(self, hours):
        self.overtime_hours += hours
        print(f"{self.name}의 야근시간 {hours}시간 추가. 총 야근시간: {self.overtime_hours}시간")
    
    def calculate_salary(self):
        base = super().calculate_salary()
        overtime_pay = self.overtime_hours * self.hourly_rate
        return base + overtime_pay
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} - 개발자 (야근시간: {self.overtime_hours}시간)"

# 직원 관리 시스템 테스트
employees = [
    Manager("김팀장", "M001", 5000000, 0.3),
    Developer("이개발", "D001", 4000000, 60000),
    Developer("박코딩", "D002", 3800000, 55000),
    Employee("최사원", "E001", 3000000)
]

# 개발자 야근시간 추가
employees[1].add_overtime(10)  # 이개발 10시간 야근
employees[2].add_overtime(15)  # 박코딩 15시간 야근

print("\n=== 직원 급여 계산 ===")
total_salary = 0
for employee in employees:
    salary = employee.calculate_salary()
    total_salary += salary
    print(f"{employee.get_info()}")
    print(f"급여: {salary:,}원\n")

print(f"총 급여 지출: {total_salary:,}원")
print(f"평균 급여: {total_salary/len(employees):,.0f}원")

In [ ]:
# 실습 문제 2: 미디어 플레이어 시스템
# MediaFile 기본 클래스와 AudioFile, VideoFile 서브클래스를 구현하세요.

class MediaFile:
    def __init__(self, filename, size_mb):
        self.filename = filename
        self._size_mb = size_mb
        self.is_playing = False
    
    @property
    def size_mb(self):
        return self._size_mb
    
    def play(self):
        if not self.is_playing:
            self.is_playing = True
            return f"{self.filename} 재생을 시작합니다."
        return f"{self.filename}는 이미 재생 중입니다."
    
    def stop(self):
        if self.is_playing:
            self.is_playing = False
            return f"{self.filename} 재생을 정지합니다."
        return f"{self.filename}는 재생 중이 아닙니다."
    
    def get_info(self):
        status = "재생 중" if self.is_playing else "정지"
        return f"{self.filename} ({self._size_mb}MB) - {status}"

class AudioFile(MediaFile):
    def __init__(self, filename, size_mb, bitrate=320):
        super().__init__(filename, size_mb)
        self.bitrate = bitrate
        self.volume = 50
    
    def set_volume(self, volume):
        self.volume = max(0, min(100, volume))
        return f"볼륨을 {self.volume}%로 설정했습니다."
    
    def play(self):
        base_message = super().play()
        if "시작" in base_message:
            return f"{base_message} (오디오 - {self.bitrate}kbps, 볼륨: {self.volume}%)"
        return base_message
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} [오디오 - {self.bitrate}kbps]"

class VideoFile(MediaFile):
    def __init__(self, filename, size_mb, resolution="1080p"):
        super().__init__(filename, size_mb)
        self.resolution = resolution
        self.subtitle_enabled = False
    
    def toggle_subtitle(self):
        self.subtitle_enabled = not self.subtitle_enabled
        status = "켜짐" if self.subtitle_enabled else "꺼짐"
        return f"자막이 {status}되었습니다."
    
    def play(self):
        base_message = super().play()
        if "시작" in base_message:
            subtitle_info = ", 자막: 켜짐" if self.subtitle_enabled else ", 자막: 꺼짐"
            return f"{base_message} (비디오 - {self.resolution}{subtitle_info})"
        return base_message
    
    def get_info(self):
        base_info = super().get_info()
        return f"{base_info} [비디오 - {self.resolution}]"

# 미디어 플레이어 테스트
media_files = [
    AudioFile("좋은노래.mp3", 8.5, 320),
    AudioFile("클래식.flac", 45.2, 1411),
    VideoFile("영화.mp4", 1500, "4K"),
    VideoFile("드라마.avi", 800, "1080p")
]

print("=== 미디어 플레이어 시스템 ===")

# 각 파일 정보 출력
for media in media_files:
    print(f"\n{media.get_info()}")

# 오디오 파일 테스트
print("\n=== 오디오 파일 테스트 ===")
audio = media_files[0]
print(audio.play())
print(audio.set_volume(75))
print(audio.stop())

# 비디오 파일 테스트
print("\n=== 비디오 파일 테스트 ===")
video = media_files[2]
print(video.toggle_subtitle())
print(video.play())
print(video.stop())

# 다형성을 활용한 일괄 재생
print("\n=== 모든 미디어 재생 ===")
for media in media_files:
    print(media.play())

print("\n=== 모든 미디어 정지 ===")
for media in media_files:
    print(media.stop())

## 8. 정리 및 요약

### 캡슐화 (Encapsulation)
- 객체의 내부 데이터와 메서드를 숨기는 기법
- `_` (보호된 속성), `__` (비공개 속성) 사용
- `@property` 데코레이터로 안전한 속성 접근 제공

### 다형성 (Polymorphism)
- 같은 인터페이스로 다른 타입의 객체들을 다루는 능력
- 메서드 오버라이딩을 통해 구현
- `super()`를 사용하여 부모 클래스의 메서드 호출

### 주요 특징
1. **코드 재사용성**: 상속과 다형성으로 코드 중복 최소화
2. **유지보수성**: 캡슐화로 안전한 데이터 관리
3. **확장성**: 새로운 클래스 추가가 용이
4. **일관성**: 동일한 인터페이스로 다양한 객체 제어

다음 장에서는 이러한 OOP 개념들을 활용한 실제 프로젝트를 구현해보겠습니다!